In [15]:
dat<- read.csv("~/Documents/Grad School/Data/Packets & Data <3/CLRN AMASE/2019-20_PilotandCohort1/Winter 20/Untouched original raw data file/for R/AMASE_W20_Cohort1T1.csv")
library(tidyverse)
library(lavaan)

In [16]:
dat %>% subset(role=="student")->dat

##sis data
SIS1<- read.csv("~/Documents/Grad School/Data/Packets & Data <3/CLRN AMASE/2019-20_PilotandCohort1/SIS data 2019 winter/SIS Data, Q1-Q3 Part 1_noheaders.csv")
SIS2<- read.csv("~/Documents/Grad School/Data/Packets & Data <3/CLRN AMASE/2019-20_PilotandCohort1/SIS data 2019 winter/SIS Data, Q1-Q3 Part 2_noheaders.csv")

SIS_all <- rbind(SIS1, SIS2)

SIS_all %>% subset(quarter ==1) -> SIS_all_Q1
colnames(SIS_all_Q1)<-paste(colnames(SIS_all_Q1),"Q1",sep="_")
SIS_all_Q1$AnonymizedStudentID <- SIS_all_Q1$AnonymizedStudentID_Q1

SIS_all %>% subset(quarter = 2) -> SIS_all_Q2
colnames(SIS_all_Q2)<-paste(colnames(SIS_all_Q2),"Q2",sep="_")
SIS_all_Q2$AnonymizedStudentID <- SIS_all_Q2$AnonymizedStudentID_Q2

SIS_all %>% subset(quarter = 3) -> SIS_all_Q3
colnames(SIS_all_Q3)<-paste(colnames(SIS_all_Q3),"Q3",sep="_")
SIS_all_Q3$AnonymizedStudentID <- SIS_all_Q3$AnonymizedStudentID_Q3

dat1 <- merge(dat,SIS_all_Q1,  by = "AnonymizedStudentID", all.x=T)
dat2 <- merge(dat1,SIS_all_Q2,  by = "AnonymizedStudentID", all.x=T)
dat3 <- merge(dat2,SIS_all_Q3,  by = "AnonymizedStudentID", all.x=T)

dat3<-dat3[!duplicated(dat3$AnonymizedStudentID), ] 

sis_2020_q4 <- read.csv("~/Documents/Grad School/Data/Packets & Data <3/CLRN SIS data/2019-2020 AY/2020 Q4 SIS Data_R.csv")
sis_2020_q4 %>% subset(school_year=="2020") ->sis_2020_q4
colnames(sis_2020_q4)<-paste(colnames(sis_2020_q4),"Q4",sep="_")
sis_2020_q4$AnonymizedStudentID <- sis_2020_q4$AnonymizedStudentID_Q4

dat<-merge(sis_2020_q4,dat3, by="AnonymizedStudentID", all.y=T)


dat<-dat[!duplicated(dat$AnonymizedStudentID), ] 

In [17]:


dat$time_fb<- dat$Q1993.2_1
dat$time_fb[dat$Q1993.1_1==2] <- 0

dat$time_insta<- dat$Q1993.2_2
dat$time_insta[dat$Q1993.1_2==2] <- 0

dat$time_snapchat<- dat$Q1993.2_3
dat$time_snapchat[dat$Q1993.1_3==2] <- 0

dat$time_twitt<- dat$Q1993.2_4
dat$time_twitt[dat$Q1993.1_4==2] <- 0

dat$time_youtb<- dat$Q1993.2_5
dat$time_youtb[dat$Q1993.1_5==2] <- 0

dat$time_tiktok<- dat$Q1993.2_6
dat$time_tiktok[dat$Q1993.1_6==2] <- 0

dat$time_othr<- dat$Q1993.2_7
dat$time_othr[dat$Q1993.1_7==2] <- 0

  
dat %>% mutate(time_total=rowSums(dplyr::select(dat,time_fb, time_insta, time_snapchat, time_twitt,
                                                time_youtb, time_tiktok, time_othr))) ->dat

#windsorize
#table(dat$time_total)
#boxplot(dat$time_total)
#summary(dat$time_total)
#IQR(dat$time_total, na.rm=T)

Tmax = 21+(1.5*13) # outlier = 1.5* greater than the IQR beyond the third quarter

dat$time_total <- ifelse(dat$time_total>Tmax,40, dat$time_total) 

dat$habitualphonecheck <- dat$Q1995

In [18]:
###LCS model with no exogenous predictor
library(lavaan)
#LCS with GPA
LCS<-'	overall_gpa_non_Q4 ~ 1*overall_gpa_non_Q3	
dGPA1 =~ 1*overall_gpa_non_Q4	
dGPA1 ~1	
overall_gpa_non_Q3 ~ 1	
overall_gpa_non_Q4 ~ 0*1	
dGPA1 ~~ dGPA1	
overall_gpa_non_Q3 ~~ overall_gpa_non_Q3	
overall_gpa_non_Q4 ~~ 0*overall_gpa_non_Q4	
dGPA1~~overall_gpa_non_Q3
'	
fitLCS <- sem(LCS, data=dat,estimator='mlr',fixed.x=FALSE,missing='fiml')
summary(fitLCS, fit.measures=TRUE, standardized=TRUE, rsquare=TRUE)

Warning message in lav_data_full(data = data, group = group, cluster = cluster, :
“lavaan WARNING: some cases are empty and will be ignored:
  249 711 821 947 1107”


lavaan 0.6-9 ended normally after 32 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                         5
                                                      
                                                  Used       Total
  Number of observations                          1545        1550
  Number of missing patterns                         3            
                                                                  
Model Test User Model:
                                              Standard      Robust
  Test Statistic                                 0.000       0.000
  Degrees of freedom                                 0           0

Model Test Baseline Model:

  Test statistic                              1122.200     517.976
  Degrees of freedom                                 1           1
  P-value                                        0.000       0.000
  Scaling

In [14]:
#LCS with GPA: predicted by phone checking during class
LCS<-'	overall_gpa_non_Q4 ~ 1*overall_gpa_non_Q3	
dGPA1 =~ 1*overall_gpa_non_Q4	
dGPA1 ~1	
overall_gpa_non_Q3 ~ 1	
overall_gpa_non_Q4 ~ 0*1	
dGPA1 ~~ dGPA1	
overall_gpa_non_Q3 ~~ overall_gpa_non_Q3	
overall_gpa_non_Q4 ~~ 0*overall_gpa_non_Q4	
dGPA1~~overall_gpa_non_Q3
dGPA1~habitualphonecheck

'	
fitLCS <- sem(LCS, data=dat,estimator='mlr',fixed.x=FALSE,missing='fiml')
summary(fitLCS, fit.measures=TRUE, standardized=TRUE, rsquare=TRUE)
modificationindices(fitLCS)

lavaan 0.6-9 ended normally after 57 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                         8
                                                      
  Number of observations                          1550
  Number of missing patterns                         5
                                                      
Model Test User Model:
                                               Standard      Robust
  Test Statistic                                117.046     104.401
  Degrees of freedom                                  1           1
  P-value (Chi-square)                            0.000       0.000
  Scaling correction factor                                   1.121
       Yuan-Bentler correction (Mplus variant)                     

Model Test Baseline Model:

  Test statistic                              1247.706     857.595
  Degrees of freedom                   

,lhs,op,rhs,mi,epc,sepc.lv,sepc.all,sepc.nox
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17,overall_gpa_non_Q3,~,overall_gpa_non_Q4,112.4046,4.27308741,4.27308741,5.0043240,5.0043240
18,overall_gpa_non_Q3,~,dGPA1,112.4914,4.27473578,32.40969828,3.5112416,3.5112416
19,overall_gpa_non_Q3,~,habitualphonecheck,112.4872,-2.30193315,-2.30193315,-0.2703424,-0.2493897
20,habitualphonecheck,~,overall_gpa_non_Q4,112.4854,-0.03742134,-0.03742134,-0.3731655,-0.3731655
21,habitualphonecheck,~,dGPA1,112.4889,0.20946394,1.58808955,1.4650053,1.4650053
22,habitualphonecheck,~,overall_gpa_non_Q3,112.4860,-0.03174925,-0.03174925,-0.2703410,-0.2703410
